In [ ]:
#Homework 3:
"""
The pipeline is created in Mage. 

The purpose of this jupyter notebook is just to explain each option selected. 
"""

In [ ]:
#Question 1: Run Mage
#0.9.70

In [ ]:
#Question 2: Creating the homework_03 project
#metadata.yaml has 55 lines.

In [ ]:
#Question 3: Creating the pipeline
#Load Stage - block to download data

import requests
from io import BytesIO
import pandas as pd

if 'data_loader' not in globals():
    from mage_ai.data_preparation.decorators import data_loader


@data_loader
def load_files(**kwargs) -> pd.DataFrame:
    #load data from the github
    response = requests.get(
        f'https://github.com/nevinpolat/mlops-zoomcamp/blob/main/week3/data/yellow_tripdata_2023-03.parquet?raw=true'
            )         

    if response.status_code != 200:
                raise Exception(response.text)

    df = pd.read_parquet(BytesIO(response.content))

    return df



In [ ]:
#Mage output: 3403766 rows x 19 columns

In [ ]:
#Question 4: Data preparation
#Transform Stage -block to clean data , remove outliers etc. 

import pandas as pd
import datetime


if 'transformer' not in globals():
    from mage_ai.data_preparation.decorators import transformer

@transformer

def clean_dataframe(df: pd.DataFrame):


    # Convert pickup and dropoff datetime columns to datetime type
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    # Calculate the trip duration in minutes
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    # Filter out trips that are less than 1 minute or more than 60 minutes

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    return df



In [ ]:
#Mage output: 3316216 rows x 20 columns

In [ ]:
#Question 5: Train a model
#Transform stage -block to create feature matrix 
#and train the model with linear regression 
#and print the y-intercept

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression


if 'transformer' not in globals():
    from mage_ai.data_preparation.decorators import transformer

@transformer
def read_dataframe(df: pd.DataFrame):

    #One hot coding
    categorical = ['PULocationID', 'DOLocationID']

    df[categorical] = df[categorical].astype(str)

    
    train_dicts = df[categorical].to_dict(orient='records')
    
    #create feature matrix 
    dv = DictVectorizer()
    X_train = dv.fit_transform(train_dicts)
    
    #Training the data  with linear regression

    target = 'duration'
    y_train = df[target].values

    lr = LinearRegression()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_train)

    print('y-Intercept is:', lr.intercept_) 

    return dv,lr

In [ ]:
#Mage output:y-Intercept is: 24.776533894980524

In [ ]:
#Question 6: Register the model
#Export Stage - block to register the linear regression model and 
#to save the dict vectorizer 

import mlflow
from mlflow.tracking import MlflowClient

import pandas as pd
import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

if 'data_exporter' not in globals():
    from mage_ai.data_preparation.decorators import data_exporter

@data_exporter
def register_model(df: pd.DataFrame,*args,**kwargs) -> None:
    
    MLFLOW_TRACKING_URI = "sqlite:///home/mlflow/mlflow.db"
    mlflow.set_experiment("LR-model")   
    mlflow.sklearn.autolog()
    
    with mlflow.start_run() as run:

        dv,lr = df
        #Save and log the artifact (dict vectorizer)
        with open("models/preprocessor.b", "wb") as f_out:
            pickle.dump(dv, f_out)

        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
       
        # Log the linear regression model and register as version 1
        mlflow.sklearn.log_model(
            sk_model=lr,
            artifact_path="sklearn-model",
            registered_model_name="linear-reg-model",
            )


In [ ]:
#Mage output:
"""
2024/06/04 15:28:29 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.

Successfully registered model 'linear-reg-model'.

Created version '1' of model 'linear-reg-model'.
"""
"""
Under mlruns > artifacts > sklearn-model > ML MODEL: 
mlflow_version: 2.12.1
model_size_bytes: 4534
model_uuid: dee4d0d3e5624baebfdc5af11cfb8ff8
run_id: 18350506568744c29479196a6946ac7d
utc_time_created: '2024-06-04 15:28:29.762933'

"""